#**XGBoost MISO**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install xgboost
#uninstall 0.90, install 1.4.2

     |████████████████████████████████| 166.7 MB 5.7 kB/s 


##Importing Libraries

In [ ]:
'''
from google.colab import files
uploaded = files.upload()
'''

import pandas as pd
import numpy as np

from google.colab import data_table
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
from numpy import absolute
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import xgboost as xgb
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
xgb.__version__

'1.4.2'

## Importing Datasets

here, the data is imported into dataframes - `df`

In [ ]:
#importing datasets
#Use workseet :alldata_allrows"

%reload_ext google.colab.data_table
df = pd.read_excel("/content/drive/MyDrive/00 MACHINE LEARNING/Tank_Data/Merged T7245,T7246.xlsx")
#df = pd.read_excel("/content/drive/MyDrive/00 MACHINE LEARNING/alldata_homogen_modev.xlsx")
df

##Missing Data

NOTE:
  This code can only run once, else it registers error
  If you wish to run again, run the previous section first

`SECTION 1` identify any columns which has missing data

`SECTION 2` is where all columns contains NaN are replaced with median


In [ ]:
##############################################################################
#                       SECTION 1 : PRELIMINARY CHECK                        #
##############################################################################


nullver = df.isnull().values.any()
sumnull = df.isnull().sum().sum()

columns_with_nan = df.columns[nullver].tolist()
print("NaN Present: ", nullver)

#The same code is in the next section to double check if all NaN are replaced



##############################################################################
# SECTION 2: REPLACING MISSING DATA WITH MEDIAN VALUE OF A PARTICULAR COLUMN #
##############################################################################

'''
-  use: df['column name'] = df['column name'].replace(['old value'],'new value')

-  Replace using median :

      median = df_i['colname'].median()
      df['colname'].fillna(median, inplace=True)

      or

       df_i['colname'].fillna(df_i['colname'].median(),inplace=True)



'''

df = df.replace(['LSWR_CDU_KV100'],np.NaN)






NaN Present:  False


##Verify Missing Data is Fixed

In [ ]:
df.fillna(df.mean(),inplace=True)
nullver = df.isnull().values.any()
sumnull = df.isnull().sum().sum()

columns_with_nan = df.columns[nullver].tolist()
print("NaN Present: ", nullver)

df


## Train Test Split

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

#Use workseet :alldata_allrows", output in columns 22-27, inputs in columns 1-21. Minor changes from yesterday.
#meaning approx rows 1-1220 for training and model development, final test data rows 1221 to last row 1700++

X = df.iloc[0:104,1:18].values
y = df.iloc[0:104,27].values

Xdep = df.iloc[104:,1:18].values
ydep = df.iloc[104:,27].values

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state = 0)

'''
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
'''

'\nX_train = sc.fit_transform(X_train)\nX_test = sc.transform(X_test)\n'

##Feature Scaling

In [ ]:

'''
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:,:] = sc.fit_transform(X_train[:,:])
X_test[:,:] = sc.transform(X_test[:,:])
'''

'\nfrom sklearn.preprocessing import StandardScaler\nsc = StandardScaler()\nX_train[:,:] = sc.fit_transform(X_train[:,:])\nX_test[:,:] = sc.transform(X_test[:,:])\n'

##Developing Model

In [ ]:

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
'''
base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=2, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None
'''
regressor = XGBRegressor()

params = {
    'n_estimators' : [100],
    'max_depth': [6],
    'learning_rate': [0.300000012], #eta
    'reg_lambda': [1],
    'colsample_bylevel'  : [1],
    'colsample_bynode'   : [1],
    'colsample_bytree'   : [1], #
    'gamma'              : [0], #
   'max_delta_step'     : [0],
    
    'min_child_weight'   : [1],
    'reg_alpha'          : [0],
    'scale_pos_weight'   : [1],
    'subsample'          : [1],
    'n_jobs'             : [2]
}

grid_regressor = GridSearchCV(estimator = regressor,
                        param_grid = params,
                        cv = 5, 
                        verbose = 1,
                        n_jobs = -1)
grid_regressor.fit(X_train,y_train)


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    6.2s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estima...
             param_grid={'colsample_bylevel': [1], 'colsample_bynode': [1],
                         'colsample_bytree': [1], 'gamma': [0],
                         'learning_rate': [0.300000012], 'max_delta_step': [0],
                         'max_depth': [6], 'min_child_weight': [1],
    

In [ ]:

print("Best Estimator: \n{}\n".format(grid_regressor.best_estimator_))
print("Best Parameters: \n{}\n".format(grid_regressor.best_params_))
print("Best Test Score: \n{}\n".format(grid_regressor.best_score_))
print("All Test Scores: \n{}\n".format(grid_regressor.cv_results_['mean_test_score']))
 
cv_result = pd.DataFrame(grid_regressor.cv_results_)


Best Estimator: 
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=2, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

Best Parameters: 
{'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.300000012, 'max_delta_step': 0, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 100, 'n_jobs': 2, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 1}

Best Test Score: 
0.14284585286013182

All Test Scores: 
[0.1428458

## Training XGBoost on Training Set

To train:

`machine_learning_model.fit(input_training_Data, output_training_data)`

To test:

`output_prediction = machine_learning_model.predict(input_test_data)`

In [ ]:
#multioutputregressor = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:linear')).fit(X, y)
regressor = XGBRegressor()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)


r2_score(y_test,y_pred)


mse = mean_squared_error(y_test, y_pred, sample_weight=None,squared=True)
rmse = mean_squared_error(y_test, y_pred, sample_weight=None,squared=False)
r2= r2_score(y_test, y_pred,sample_weight=None)


print('MSE Value: {:.5f}'.format(mse))
print('RMSE Value: {:.5f}'.format(rmse))
print('R-squared Value: {:.2f}'.format(r2))

MSE Value: 6.97618
RMSE Value: 2.64125
R-squared Value: 0.64


##Plotting Predicted outpus vs. Actual Data (Test Set)

Uncomment if you wish to plot Test Data vs. Predicted Data

In [ ]:
#print ypred and ytest in a single plot

plt.plot(y_test,'#0474BE', y_pred,'#D95319')
plt.legend(('Actual Data','Predicted Data'),
           loc='lower right', shadow=True )
plt.title('Abs wax')
#plt.savefig('/content/drive/MyDrive/00 MACHINE LEARNING/Tank_Data/Base Model T7249,T7250/Plot/Sim_95',dpi=300)
plt.show()

##Save Model

In [ ]:

from xgboost import Booster

regressor.save_model('')






##Deploy

In [ ]:
y_dep = regressor.predict(Xdep)
plt.plot(ydep,'#0474BE', y_dep,'#D95319')
plt.legend(('Actual Data','Predicted Data'),
           loc='lower right', shadow=True, bbox_to_anchor=(1.05, 1))
plt.title('MABP')
plt.show()